In [1]:
from tensorflow.keras.utils import normalize
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate
import numpy as np
from tqdm import tqdm 
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam

In [3]:
model = load_model(r".\UNET_Model_Weight.h5")

x = model.output
new_model = Model(inputs=model.input, outputs=x)
new_model.compile(optimizer=Adam(learning_rate=3.5000e-04), loss="binary_crossentropy", metrics=["accuracy"])
new_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 256, 256, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 256, 256, 64)      │           1,792 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 256, 256, 64)      │             256 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 256, 256, 64)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 256, 256, 64)      │          36,928 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 256, 256, 64)      │             256 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 256, 256, 64)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 128, 128, 64)      │               0 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 128, 128, 128)     │          73,856 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 128, 128, 128)     │             512 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 128, 128, 128)     │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 128, 128, 128)     │         147,584 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 128, 128, 128)     │             512 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (None, 128, 128, 128)     │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_1               │ (None, 64, 64, 128)       │               0 │ activation_3[0][0]         │
│ (MaxPooling2D)                │                           │               

 Total params: 31,055,297 (118.47 MB)

 Trainable params: 31,043,521 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [5]:
def load_images(imgsort, masksort, image_dir, mask_dir):
    images, masks = [],[]
    slash = "\\"
    for img, msk in tqdm(zip(imgsort, masksort), total=len(imgsort), desc = 'Loading Images and Masks'):
        image_string = image_dir + slash + img
        image = cv2.imread(image_string, cv2.IMREAD_COLOR)
        mask_string = mask_dir + slash + msk
        mask = cv2.imread(mask_string, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (256,256))
        mask = cv2.resize(mask, (256,256))
        images.append(image)
        masks.append(mask)

        del image, mask
    return images, masks

In [7]:
# Loading in the images and masks
import os
map_dir = r'./CoralReef_Map'
mask_dir = r'./CoralReef_Mask'
map_sort = os.listdir(map_dir)
mask_sort = os.listdir(mask_dir)
I,M = load_images(map_sort, mask_sort, map_dir, mask_dir)

Loading Images and Masks: 100%|█████████████████████████████████████████████████████| 250/250 [00:02<00:00, 104.59it/s]


In [9]:
I = np.array(I)
I = np.expand_dims(I,-1)
M = np.array(M)
M = np.expand_dims(M,-1)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(I, M, test_size = 0.25, random_state = 0)

In [13]:
X_test = np.squeeze(X_test)
X_train = np.squeeze(X_train)

In [15]:
IMG_HEIGHT = I.shape[1]
IMG_WIDTH = I.shape[2]
IMG_CHANNELS = I.shape[3]
print(IMG_HEIGHT,IMG_WIDTH, IMG_CHANNELS)

256 256 3


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
seed=24
img_data_gen_args = dict(rotation_range=90,
                     width_shift_range=0.3,
                     height_shift_range=0.3,
                     shear_range=0.5,
                     zoom_range=0.3,
                     horizontal_flip=True,
                     vertical_flip=True,
                     fill_mode='reflect')
mask_data_gen_args = dict(rotation_range=90,
                     width_shift_range=0.3,
                     height_shift_range=0.3,
                     shear_range=0.5,
                     zoom_range=0.3,
                     horizontal_flip=True,
                     vertical_flip=True,
                     fill_mode='reflect',
                    preprocessing_function = lambda x: np.where(x>147, 1, 0).astype(x.dtype))

image_data_generator = ImageDataGenerator(**img_data_gen_args)
batch_size = 8

image_generator = image_data_generator.flow(X_train, seed=seed, batch_size=batch_size)
valid_img_generator = image_data_generator.flow(X_test, seed=seed, batch_size=batch_size)
mask_data_generator = ImageDataGenerator(**mask_data_gen_args)
mask_generator = mask_data_generator.flow(y_train, seed=seed, batch_size=batch_size)
valid_mask_generator = mask_data_generator.flow(y_test, seed=seed, batch_size=batch_size)

def my_image_mask_generator(image_generator, mask_generator):
    train_generator = zip(image_generator, mask_generator)
    for (img, mask) in train_generator:
        yield (img, mask)

my_generator = my_image_mask_generator(image_generator, mask_generator)

validation_datagen = my_image_mask_generator(valid_img_generator, valid_mask_generator)
 
steps_per_epoch = 3*(len(X_train))//batch_size

lr_schedule = ReduceLROnPlateau(monitor="val_loss", factor=0.2,   patience=3, verbose=1)
callbacks = [lr_schedule]
history = new_model.fit(my_generator, validation_data=validation_datagen, 
                    steps_per_epoch=steps_per_epoch, 
                    validation_steps=steps_per_epoch, epochs=10, callbacks =callbacks)

Epoch 1/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 826s 11s/step - accuracy: 0.9281 - loss: 0.1776 - val_accuracy: 0.9207 - val_loss: 0.1979 - learning_rate: 3.5000e-04
Epoch 2/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 797s 11s/step - accuracy: 0.9278 - loss: 0.1792 - val_accuracy: 0.9194 - val_loss: 0.1980 - learning_rate: 3.5000e-04
Epoch 3/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 788s 11s/step - accuracy: 0.9297 - loss: 0.1734 - val_accuracy: 0.9167 - val_loss: 0.2060 - learning_rate: 3.5000e-04
Epoch 4/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.9269 - loss: 0.1804
Epoch 4: ReduceLROnPlateau reducing learning rate to 6.99999975040555e-05.
70/70 ━━━━━━━━━━━━━━━━━━━━ 705s 10s/step - accuracy: 0.9269 - loss: 0.1804 - val_accuracy: 0.9083 - val_loss: 0.2263 - learning_rate: 3.5000e-04
Epoch 5/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 499s 7s/step - accuracy: 0.9289 - loss: 0.1770 - val_accuracy: 0.9214 - val_loss: 0.1931 - learning_rate: 7.0000e-05
Epoch 6/10
17/70 ━━━━━━━━━━━━━━━━━━━━ 5:20 6s/step - accuracy: 0.9252 - lo

In [14]:
from tensorflow.keras.models import load_model
model.save(r".\Updated_model.h5")
